In [18]:
# install packages
%pip install -q gdown
%pip install --upgrade --quiet \
    pandas==2.2.2 \
    xgboost==2.0.3 \
    scikit-learn==1.3.2 
%pip install pandas openpyxl


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
# import statements
from datetime import *
import pandas as pd
import xgboost as xgb

from pathlib import Path
base_path = Path.home() / "Documents" / "totalenergies_price_forecasting"

In [21]:
def get_feat(file_path):
    df = pd.read_csv(file_path)
    X = df.loc[:, df.columns != 'Price ($/MWh)'] 
    y = df['Price ($/MWh)']

    return X, y

In [15]:
def feat_forecast(feature, X, y, fh, ml): 
  print(f"Training univariate forecaster for feature: {feature}")
   
  # create lagged versions of the feature
  for lag in range(1, ml + 1):
    X[f"{feature}_lag{lag}"] = X[feature].shift(lag)
     
  feature_lagged = X[[f"{feature}_lag{lag}" for lag in range(1, ml + 1)]].dropna()  # drop NaNs
  y_target = X[feature].shift(-fh).dropna()
     
  # align X and y (must use common indices)
  common_idx = feature_lagged.index.intersection(y_target.index)
  X_feat = feature_lagged.loc[common_idx]
  y_feat = y_target.loc[common_idx]

  train_size = int(0.8 * len(X_feat))
  X_feat_train, X_feat_test = X_feat.iloc[:train_size], X_feat.iloc[train_size:]
  y_feat_train, y_feat_test = y_feat.iloc[:train_size], y_feat.iloc[train_size:]

  model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=50) 
  model.fit(X_feat_train, y_feat_train)  # train model

  # predict future forecast_horizon steps using most recent lag features
  last_known = X[[f"{feature}_lag{lag}" for lag in range(1, ml + 1)]].iloc[-fh:]

  feature_preds = model.predict(last_known)  # make prediction
  return feature_preds

In [23]:
def time_shift(X, y, fh, discard):
    y_shifted = y.shift(-fh)
    # drop rows with NaNs caused by shift (they occur at the end)
    valid_idx = y_shifted.dropna().index

    # align X and y so that input X[t] corresponds to output y[t+horizon]
    X_supervised = X.loc[valid_idx]
    for col in discard:
        X_supervised = X_supervised.drop(col, axis=1)
    y_supervised = y_shifted.loc[valid_idx]

    # Print the shapes to confirm
    print(f"Shape of X_supervised: {X_supervised.shape}")
    print(f"Shape of y_supervised: {y_supervised.shape}")

    # Optional: Reset index if you want a clean DataFrame
    X_supervised = X_supervised.reset_index(drop=True)
    y_supervised = y_supervised.reset_index(drop=True)

    return X_supervised, y_supervised

In [27]:
def hybrid_feature_forecast(file_path, dynamic_features, forecast_horizon, max_lag):
    X, y = get_feat(file_path)
    predicted_features = pd.DataFrame() 

    for feat in dynamic_features:
        feature_preds = feat_forecast(feat, X, y, forecast_horizon, max_lag)
        predicted_features[feat] = feature_preds
    
    X_supervised, y_supervised = time_shift(X, y, forecast_horizon, dynamic_features)
    print(X_supervised)
    print(predicted_features)
    
    # Add suffix to predicted features to indicate they are forecasts
    predicted_features = predicted_features.add_suffix('_forecast')

    # Concatenate original X_supervised with forecasted weather features
    X_combined = pd.concat([X_supervised, predicted_features], axis=1)

    return X_combined, y_supervised

In [28]:
forecast_horizon = 48
max_lag = 24

file_path = Path.home() / "Downloads" / "All_Years_Fuel_and_RTM.csv"
dynamic_features = ['Biomass', 'Coal']

X_combined, y_supervised = hybrid_feature_forecast(file_path, dynamic_features, forecast_horizon, max_lag)
X_combined.head()
y_supervised.head()

Training univariate forecaster for feature: Biomass
Training univariate forecaster for feature: Coal
Shape of X_supervised: (26250, 52)
Shape of y_supervised: (26250,)
                 Timestamp         Gas      Hydro      Nuclear  Biomass_lag1  \
0      2022-01-01 01:00:00  331.021529   6.642016  1269.765473           NaN   
1      2022-01-01 02:00:00  187.038593   1.623843  1269.875205      5.577449   
2      2022-01-01 03:00:00  127.970148   1.636805  1269.981095      5.576623   
3      2022-01-01 04:00:00  126.314995   1.601775  1269.741637      5.579368   
4      2022-01-01 05:00:00  124.026816   1.565176  1269.762159      5.578247   
...                    ...         ...        ...          ...           ...   
26245  2024-12-29 19:00:00  806.553019  49.847791  1269.866314      3.334973   
26246  2024-12-29 20:00:00  600.870198  33.254741  1269.642755      3.332221   
26247  2024-12-29 21:00:00  331.148897   2.816440  1269.719207      3.327587   
26248  2024-12-29 22:00:00  313.

0    34.9875
1    34.9050
2    35.6700
3    36.3500
4    37.0725
Name: Price ($/MWh), dtype: float64